<a href="https://colab.research.google.com/github/ofzlo/WBC-AI-Imageing/blob/main/AI_Cell_Imaging_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab.patches import cv2_imshow
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

# Image Preprocessing

In [ ]:
def isText(img):
  colors = ['b', 'g', 'r']
  bgr_planes = cv2.split(img)
  global green

  for (p, c) in zip(bgr_planes, colors):
    if c == 'g':
      green = cv2.calcHist([p], [0], None, [256], [0, 256])

  if green[250] > 0:
    return True
  
  return False

In [ ]:
def img_crop(img, path):
  cv2.imwrite(img, path)

In [ ]:
def find_contours(img, size):
  RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  gray = cv2.cvtColor(RGB, cv2.COLOR_RGB2GRAY)
  res, thr = cv2.threshold(gray, size, 255, cv2.THRESH_BINARY)
  contours, _ = cv2.findContours(thr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  
  ct = []
  for i in range(len(contours)):
    cnt = contours[i]
    x, y, w, h = cv2.boundingRect(cnt)
    test_img = img[y:y+h, x:x+w]
    if w+h > 4000 or w+h < 80 or isText(test_img):
      continue
    else:
      ct.append(cnt)
  
  return ct

In [ ]:
def color_distribution(img):
  colors = ['b', 'g', 'r']
  bgr_planes = cv2.split(img)
  global blue

  for (p, c) in zip(bgr_planes, colors):
    if c == 'b':
      blue = cv2.calcHist([p], [0], None, [256], [0, 256])
    hist = cv2.calcHist([p], [0], None, [256], [0, 256])

  tmp1 = blue[195].tolist()
  tmp2 = int(tmp1[0])
  return tmp2

In [ ]:
def draw_bb(img, contours):
  num=0
  for i in range(len(contours[:]) - 1):
    cnt = contours[i]
    x, y, w, h = cv2.boundingRect(cnt)
    cv2.rectangle(img, (x, y), (x+w, y+h), (0,0,255), 3)
    #path 지정해주기
    CROP_PATH = ''
    path = CROP_PATH + str(num)+'.jpg'
    img_crop(path, img[y:y+h, x:x+w])
    num += 1
  cv2_imshow(img)

In [ ]:
def erode(img):
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
  erosion = cv2.erode(img,kernel,iterations=2)
  return erosion

In [ ]:
def dilate(img):
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
  dilation = cv2.dilate(img,kernel,iterations=3)
  return dilation

In [ ]:
def find_cells(img):

  img_copy = img.copy()

  isTrue = True

  #핵 주변 색 분포 파악
  while(isTrue):

    #핵 찾기
    contours = find_contours(img, 100)

    #핵 주변 찾기
    for i in range(len(contours) - 1):
      cnt = contours[i]
      x, y, w, h = cv2.boundingRect(cnt)
      img_cut = img_copy[y-h:y+2*h, x-w:x+2*w]

      #색 분포 정상
      if color_distribution(img_cut) < 300:
          isTrue = False

      #색 분포 비정상
      else:
        try:
          isTrue = True
          img_copy[y-h:y+2*h, x-w:x+2*w] = erode(dilate(img_cut))
        except:
          isTrue = False

  #세포질 찾기
  contours = find_contours(img_copy, 190)

  #바운딩 박스 그리기
  draw_bb(img, contours)
  

In [ ]:
IMG = ''
img=cv2.imread(IMG)
find_cells(img)

# Clustering

In [ ]:
import glob
import numpy as np 
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow

#kmeans library 
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score 

data=[]
CROP_SAVE_DIR = ''
files=glob.glob(CROP_SAVE_DIR)
for my_file in files:
  #print(my_file)
  image_bgr=cv2.imread(my_file)
  image=cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB) #rgb로 바꾸기
  image=cv2.resize(image, (64,64))
  image=np.array(image)
  data.append(image)

In [ ]:
img_np=np.array(data)
print(img_np.shape)

RESIZE_SAVE_DIR = ''
np.save(RESIZE_SAVE_DIR,img_np)

In [ ]:
NPY_DIR = ''
data=np.load(NPY_DIR)
data_2d=data.reshape(-1,64*64*3)

#print("sample_data의 shape 맞추기:", data)
#print("sample_data_2d의 shape 맞추기:", data_2d)

print("sample_data.shape:", data.shape)
print("sample_data_2d.shape:", data_2d.shape)

In [ ]:
km=KMeans(n_clusters=6, random_state=42)
km.fit(data_2d)
print('km.labels_', km.labels_)
print('km.labels_.shape', km.labels_.shape)
print(km.labels_[0:])

print(np.unique(km.labels_, return_counts=True))
print(km.labels_)

import matplotlib.pyplot as plt

def draw_sample_data(arr, ratio=1):
    n = len(arr)  # n은 샘플 개수
    # 한 줄에 10개씩 이미지를 그림. 샘플 개수를 10으로 나누어 전체 행 개수를 계산.
    print(n)
    rows = int(np.ceil(n / 10))
    # 행이 1개 이면 열 개수는 샘플 개수. 그렇지 않으면 10개.
    cols = n if rows < 2 else 10
    fig, axs = plt.subplots(rows, cols,
                            figsize=(cols * ratio, rows * ratio), squeeze=False)
    for i in range(rows):
        #print(i)
        for j in range(cols):
            if i * 10 + j < n:  # n 개까지.
                #print(i)
                axs[i, j].imshow(arr[i * 10 + j], cmap='gray_r')
            axs[i, j].axis('off')
    plt.show()

### Clustering Result

In [ ]:
draw_sample_data(data[km.labels_ == 0])

In [ ]:
draw_sample_data(data[km.labels_ == 1])

In [ ]:
draw_sample_data(data[km.labels_ == 2])

In [ ]:
draw_sample_data(data[km.labels_ == 3])

In [ ]:
draw_sample_data(data[km.labels_ == 4])

In [ ]:
draw_sample_data(data[km.labels_ == 5])

# Modeling
Model Summary : 'MobileNetV2'

In [ ]:
from keras.models import load_model
import os

In [ ]:
MODEL_PATH = ''
model = load_model(MODEL_PATH)

In [ ]:
TEST_DIR = ''
result = [0, 0, 0, 0, 0, 0]
file_list = os.listdir(TEST_DIR)

for i in range(len(file_list)):
  src = cv2.imread(os.path.join(TEST_DIR, file_list[i]))
  img = cv2.resize(src,(96, 96))
  img = np.expand_dims(img, axis=0)
  pred_labels = model.predict(img)
  class_idx = np.argmax(pred_labels)
  result[class_idx] += 1

print_result(result)